# Total Cases on Ships

In [ ]:
#imports
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import folium

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

%matplotlib inline

import math
import random
from datetime import timedelta 

import warnings
warnings.filterwarnings('ignore')

#color pallette 
cnf = '#393e46'
dth = '#ff2e63'
rec = '#21bf73'
act = '#fe9801'

In [ ]:
import plotly as py 
py.offline.init_notebook_mode(connected = True)

In [ ]:
import os

In [ ]:
try:
    os.system("rm -rf Covid-19-Preprocessed-Dataset")
except:
    print('File does not exist')

In [ ]:
!git clone https://github.com/laxmimerit/Covid-19-Preprocessed-Dataset

In [ ]:
df = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/covid_19_data_cleaned.csv',parse_dates=['Date'])

country_daywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/country_daywise.csv',parse_dates=['Date'])
countrywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/countrywise.csv')
daywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/daywise.csv',parse_dates=['Date'])

In [ ]:
df['Province/State'] = df['Province/State'].fillna("")
df.head()

In [ ]:
# Ships 
# ======

ship_rows = df['Province/State'].str.contains('Grand Princess') | df['Province/State'].str.contains('Diamond Princess') | df['Country'].str.contains('Grand Princess') | df['Country'].str.contains('Diamond Princess') | df['Country'].str.contains('MS Zaandam')
ship = df[ship_rows]

df = df[~ship_rows]

In [ ]:
ship_latest = ship[ship['Date'] == max(ship['Date'])]
ship_latest

In [ ]:
ship_latest.style.background_gradient(cmap = 'Pastel1_r')

# Cases Over The Time with Area Plot

In [ ]:
temp = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index
#temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)
#temp

In [ ]:
tm = df.melt(id_vars = 'Date', value_vars = ['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm , path = ['variable'], values= 'value', height = 250, width = 800, color_discrete_sequence=[act, rec, dth])
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
temp = df.groupby('Date').mean()
temp

In [ ]:
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum()
temp

In [ ]:
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = df.melt(id_vars = 'Date', value_vars = ['Recovered', 'Deaths', 'Active'], var_name = 'Case', value_name = 'Count')
temp

Since the number of recoveries as of August 5 is not included in the data set, it appears as 0 as of August 5, 2021, which has improved in the graphics and tables.

In [ ]:
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = df.melt(id_vars = 'Date', value_vars = ['Recovered', 'Deaths', 'Active'], var_name = 'Case', value_name = 'Count')
fig = px.area(temp, x = 'Date', y = 'Count', color = 'Case', height = 600, title = 'Cases over time', color_discrete_sequence=[rec, dth, act])

fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

# Folium Maps

In [ ]:
temp = df[df['Date']==max(df['Date'])]
temp

In [ ]:
temp = df[df['Date']== max (df['Date'])]

m= folium.Map(location=[0,0], tiles = 'cartodbpositron', min_zoom = 1, max_zoom=4, zoom_start=1 )

for i in range(0, len(temp)):
    folium.Circle(location=[temp.iloc[i]['Lat'], temp.iloc[i]['Long']], color = 'crimson', fill = 'crimson',
                 toolip = '<li><bold> Country:  ' + str(temp.iloc[i]['Country'])+ 
                           '<li><bold> Province : '+ str(temp.iloc[i]['Province/State'])+
                          '<li><bold> Confirmed:  ' + str(temp.iloc[i]['Confirmed'])+
                          '<li><bold> Deaths:  ' + str(temp.iloc[i]['Deaths']),
                 radius = int(temp.iloc[i]['Confirmed'])**0.5).add_to(m)
m

# Confirmed Cases with Choropleth Map

In [ ]:
country_daywise.head()

In [ ]:
fig = px.choropleth(country_daywise, locations= 'Country', locationmode='country names', color = country_daywise['Confirmed'],
                   hover_name = 'Country', animation_frame=country_daywise['Date'].dt.strftime('%Y-%m-%d'),
                   title='Cases over time', color_continuous_scale=px.colors.sequential.Inferno)
fig.update(layout_coloraxis_showscale = True)
fig.show()

# Deaths and Recoveries per 100 Cases

In [ ]:
daywise.head()

In [ ]:
fig_c = px.bar(daywise, x = 'Date', y = 'Confirmed', color_discrete_sequence=[act])
fig_d = px.bar(daywise, x = 'Date', y = 'Deaths', color_discrete_sequence=[dth])

fig = make_subplots(rows = 1, cols = 2, shared_xaxes=False, horizontal_spacing=0.1,
                   subplot_titles=('Confirmed Cases','Death Cases'))

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.update_layout(height = 480)

fig.show()

# Confirmed and Death Cases with Static Colormap

In [ ]:
fig_c = px.choropleth(countrywise, locations='Country',locationmode='country names',
                     color = np.log(countrywise['Confirmed']), hover_name = 'Country',
                     hover_data = ['Confirmed'])
temp= countrywise[countrywise['Deaths']>0]
fig_d = px.choropleth(temp, locations='Country',locationmode='country names',
                     color = np.log(temp['Deaths']), hover_name = 'Country',
                     hover_data = ['Deaths'])

fig = make_subplots(rows = 1, cols = 2, subplot_titles= ['Confirmed','Deaths'],
                   specs = [[{'type': 'choropleth'},{'type':'choropleth'}]])

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.show()

In [ ]:
daywise.columns

In [ ]:
fig1 = px.line(daywise, x = 'Date', y = 'Deaths / 100 Cases', color_discrete_sequence=[dth])
fig2 = px.line(daywise, x = 'Date', y = 'Recovered / 100 Cases', color_discrete_sequence=[rec])
fig3 = px.line(daywise, x = 'Date', y = 'Deaths / 100 Recovered', color_discrete_sequence=[dth])

fig = make_subplots(rows = 1, cols = 3, shared_xaxes=False,
                   subplot_titles=('Deaths / 100 Cases', 'Recovered / 100 Cases','Deaths / 100 Recovered'))

fig.add_trace(fig1['data'][0], row = 1, col = 1)
fig.add_trace(fig2['data'][0], row = 1, col = 2)
fig.add_trace(fig3['data'][0], row = 1, col = 3)

fig.update_layout(height = 400)
fig.show()